# 📄 步骤 1：PDF 文本提取

这个 notebook 负责：
1. 从 PDF 文件中提取文本
2. 清洗和规范化文本
3. 分块处理（为 LLM 准备）
4. 缓存结果（避免重复提取）

**⏱️ 预计时间**：取决于 PDF 数量和大小，通常 2-10 分钟

## 1. 加载环境配置

In [ ]:
import os
import sys
import yaml
from pathlib import Path

# 读取环境变量
if os.path.exists('/content/PWD_Project/.env'):
    with open('/content/PWD_Project/.env', 'r') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

PROJECT_ROOT = os.environ.get('PROJECT_ROOT', '/content/PWD_Project')
CONFIG_PATH = os.environ.get('CONFIG_PATH', f'{PROJECT_ROOT}/config/config_colab.yaml')

# 切换到项目目录
os.chdir(PROJECT_ROOT)
sys.path.insert(0, f'{PROJECT_ROOT}/modules')

# 加载配置
with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
    CONFIG = yaml.safe_load(f)

print(f"✅ 项目根目录: {PROJECT_ROOT}")
print(f"✅ 配置文件: {CONFIG_PATH}")
print(f"✅ PDF 目录: {CONFIG['pdf']['input_directory']}")

## 2. 导入必要的库

In [ ]:
import re
import json
import logging
from typing import Dict, List
from datetime import datetime

import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('PDF_Extractor')

print("✅ 库导入完成")

## 3. PDF 提取器类

In [ ]:
class PDFExtractor:
    """PDF 文本提取器（Colab 版本）"""
    
    def __init__(self, cache_dir: str = None):
        self.cache_dir = cache_dir or f"{PROJECT_ROOT}/cache"
        os.makedirs(self.cache_dir, exist_ok=True)
    
    def extract_from_directory(self, pdf_dir: str) -> Dict[str, str]:
        """从目录提取所有 PDF 文本"""
        if not os.path.exists(pdf_dir):
            logger.error(f"PDF 目录不存在: {pdf_dir}")
            return {}
        
        pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
        
        if not pdf_files:
            logger.warning(f"未找到 PDF 文件: {pdf_dir}")
            return {}
        
        logger.info(f"找到 {len(pdf_files)} 个 PDF 文件")
        
        pdf_texts = {}
        for pdf_file in tqdm(pdf_files, desc="提取 PDF"):
            pdf_path = os.path.join(pdf_dir, pdf_file)
            text = self._extract_text(pdf_path, pdf_file)
            if text:
                pdf_texts[pdf_file] = text
        
        logger.info(f"成功提取 {len(pdf_texts)} 个 PDF")
        return pdf_texts
    
    def _extract_text(self, pdf_path: str, pdf_name: str) -> str:
        """提取单个 PDF 的文本"""
        # 检查缓存
        cache_file = os.path.join(self.cache_dir, f"{Path(pdf_name).stem}.txt")
        if os.path.exists(cache_file):
            logger.info(f"从缓存读取: {pdf_name}")
            with open(cache_file, 'r', encoding='utf-8') as f:
                return f.read()
        
        try:
            doc = fitz.open(pdf_path)
            text_parts = []
            
            for page_num, page in enumerate(doc, 1):
                text = page.get_text()
                if text.strip():
                    text_parts.append(text)
            
            doc.close()
            
            full_text = '\n'.join(text_parts)
            full_text = self._clean_text(full_text)
            
            # 保存到缓存
            with open(cache_file, 'w', encoding='utf-8') as f:
                f.write(full_text)
            
            logger.info(f"提取完成: {pdf_name} ({len(full_text)} 字符)")
            return full_text
            
        except Exception as e:
            logger.error(f"提取失败 {pdf_name}: {e}")
            return ""
    
    def _clean_text(self, text: str) -> str:
        """清洗文本"""
        # 移除多余空白
        text = re.sub(r'\s+', ' ', text)
        # 移除控制字符
        text = re.sub(r'[\x00-\x08\x0b-\x0c\x0e-\x1f\x7f-\x9f]', '', text)
        # 移除页码（可选）
        text = re.sub(r'\n\d+\n', '\n', text)
        return text.strip()

print("✅ PDFExtractor 类定义完成")

## 4. 执行 PDF 文本提取

In [ ]:
# 创建提取器
extractor = PDFExtractor(cache_dir=CONFIG['output']['cache_directory'])

# 提取文本
print("\n" + "="*60)
print("开始提取 PDF 文本")
print("="*60 + "\n")

start_time = datetime.now()

pdf_texts = extractor.extract_from_directory(CONFIG['pdf']['input_directory'])

end_time = datetime.now()
duration = end_time - start_time

print("\n" + "="*60)
print("PDF 提取完成")
print("="*60)
print(f"\n耗时: {duration}")
print(f"提取的 PDF 数量: {len(pdf_texts)}")
print(f"\n总字符数: {sum(len(text) for text in pdf_texts.values()):,}")

## 5. 文本分块

将长文本分割成适合 LLM 处理的块

In [ ]:
def create_chunks(pdf_texts: Dict[str, str], 
                  chunk_size: int = 2000, 
                  overlap: int = 200) -> List[Dict]:
    """将文本分块"""
    chunks = []
    
    for pdf_name, text in pdf_texts.items():
        # 按块大小切分
        for i in range(0, len(text), chunk_size - overlap):
            chunk_text = text[i:i + chunk_size]
            
            # 跳过过短的块
            if len(chunk_text.strip()) < 100:
                continue
            
            chunks.append({
                'chunk_id': f"{Path(pdf_name).stem}_{len(chunks)}",
                'source_pdf': pdf_name,
                'text': chunk_text,
                'concepts': []  # 为后续步骤准备
            })
    
    return chunks

# 执行分块
chunks = create_chunks(pdf_texts, chunk_size=2000, overlap=200)

print(f"\n✅ 创建了 {len(chunks)} 个文本块")
print(f"\n平均块大小: {sum(len(c['text']) for c in chunks) // len(chunks)} 字符")

## 6. 查看样例

In [ ]:
# 显示第一个文本块的样例
if chunks:
    sample = chunks[0]
    print("\n📄 样例文本块:")
    print(f"\nChunk ID: {sample['chunk_id']}")
    print(f"来源 PDF: {sample['source_pdf']}")
    print(f"文本长度: {len(sample['text'])} 字符")
    print("\n文本预览:")
    print("-" * 60)
    print(sample['text'][:500] + "...")
    print("-" * 60)

## 7. 保存结果

In [ ]:
# 保存文本和块信息
output_dir = CONFIG['output']['base_directory']

# 保存 PDF 文本
texts_file = f"{output_dir}/pdf_texts.json"
with open(texts_file, 'w', encoding='utf-8') as f:
    json.dump(pdf_texts, f, ensure_ascii=False, indent=2)

# 保存文本块
chunks_file = f"{output_dir}/text_chunks.json"
with open(chunks_file, 'w', encoding='utf-8') as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

# 保存统计信息
stats = {
    'pdf_count': len(pdf_texts),
    'chunk_count': len(chunks),
    'total_chars': sum(len(text) for text in pdf_texts.values()),
    'avg_chunk_size': sum(len(c['text']) for c in chunks) // len(chunks) if chunks else 0,
    'extraction_time': str(duration),
    'timestamp': datetime.now().isoformat()
}

stats_file = f"{output_dir}/extraction_stats.json"
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print(f"\n✅ 结果已保存:")
print(f"   📄 PDF 文本: {texts_file}")
print(f"   📦 文本块: {chunks_file}")
print(f"   📊 统计信息: {stats_file}")

## 8. PDF 提取完成

### ✅ 完成情况

- 提取的 PDF 数量
- 生成的文本块数量
- 缓存文件位置

### 📝 下一步

请打开 **`02_概念提取.ipynb`** 继续执行概念提取

### 💡 提示

- 提取的文本已缓存，重新运行不会重复提取
- 如需重新提取，删除 `cache/` 目录下的 `.txt` 文件
- 可以随时添加新的 PDF 文件到 PDF 目录

In [ ]:
# 显示下一步提示
print("\n" + "="*60)
print("🎉 PDF 提取步骤完成！")
print("="*60)
print(f"\n📊 统计:")
print(f"   - PDF 文件: {stats['pdf_count']} 个")
print(f"   - 文本块: {stats['chunk_count']} 个")
print(f"   - 总字符数: {stats['total_chars']:,} 个")
print(f"   - 耗时: {stats['extraction_time']}")
print(f"\n📁 文件保存在: {output_dir}")
print(f"\n➡️  下一步: 打开 02_概念提取.ipynb")